In [1]:
#Importo las librerias
import pandas as pd
import requests
import progressbar
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
from open_sheet import open_sheet

In [2]:
scope_gdoc = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
cre = ServiceAccountCredentials.from_json_keyfile_name('', scope_gdoc)
gc = gspread.authorize(cre)

In [3]:
sheet = 'listados_de_productos_con_incongruencias'
Planilla = '1NjNi3oEqXr8bF7ySDghnzzpzayUywVxFYuYTbVRDHjo'

df = open_sheet(sheet,
                Planilla,
                gc)
df = df[df['message']!='picture'].reset_index(drop=True)
df = df[['id','Domain']]

In [5]:
def get_attr_templates(dom):
    s = requests.Session()
    url = "https://internal-api.mercadolibre.com/internal/pdp-monitor/domain/attributes-used-template/"+str(dom)
    headers = { "Content-Type": "application/json"}
    response = s.get(url, headers=headers)
    s.close()
    return response

def get_attr_prd(prd):
    s = requests.Session()
    url = "https://internal-api.mercadolibre.com/catalog_products/"+str(prd)
    headers = { "Content-Type": "application/json"}
    response = s.get(url, headers=headers)
    s.close()
    return response

In [8]:
sites = ['MLA', 'MLB','MLM']

df_list = []
for i in progressbar.progressbar(range(len(df))):
    for s in sites:
        prod = s+df.loc[i,'id']
        dom = df.loc[i,'Domain']

        attr_template = get_attr_templates(dom)
        attr_prod = get_attr_prd(prod)
        if attr_template.status_code in range(200,300) and attr_prod.status_code in range(200,300):
            attr_template= attr_template.json()
            attr_prod = attr_prod.json().get('attributes')

            list_attr_t=[]
            for t in range(len(attr_template)):
                list_attr_t.append(attr_template[t].get('id'))

            list_attr_prod=[]
            empty = []
            for p in range(len(attr_prod)):
                attr_p = attr_prod[p].get('id')
                list_attr_prod.append(attr_p)
                
                if attr_prod[p].get('value_name') == '':
                    empty_attr.append(attr_prod[p].get('id'))


            attr_used = [a for a in list_attr_prod if a in list_attr_t]
            not_used = [a for a in list_attr_t if a not in list_attr_prod]        
            
            df_list.append({
                'id':df.loc[i,'id'],
                'prod':prod,
                'site':s,
                'Domain':dom,
                'q_used': len(attr_used),
                'q_not_used':len(not_used),
                'attr_used':attr_used,
                'attr_not_used':not_used,
                'empty': empty
            })

100% (2106 of 2106) |####################| Elapsed Time: 3:32:53 Time:  3:32:53


In [9]:
df_upload = pd.DataFrame(df_list)
df_upload

In [13]:
Planilla = '1NjNi3oEqXr8bF7ySDghnzzpzayUywVxFYuYTbVRDHjo'
d2g.upload(df_upload, Planilla, 'listado_incongruencias', credentials=cre)

<Worksheet 'listado_incongruencias' id:787862058>